# Exploitation 0001 - January 06 26

## Goal: I am creating an agent that I will exploit.

This agent will take in a resume and recommend what the resume owner needs to learn to become relevant in the AI Era.

To start, I will install initialize my LLM

In [3]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') or getpass('Enter Open Ai API KEY')

In [4]:
from langchain_openai import ChatOpenAI

openai_model = 'gpt-4o-mini'
llm = ChatOpenAI(temperature=0.0, model=openai_model)

creative_llm = ChatOpenAI(temperature=0.9, model=openai_model)

The next thing we will do is to prepare our prompt

In [11]:
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

resume = '''
Experience: 6+ years as a backend or full-stack engineer building web applications
Python/Django: 3+ years of experience with Python and Django
Frameworks: Django REST Framework and/or django-ninja
Scalability: Proven experience running Django at scale
Asynchronous Systems: Celery, RabbitMQ, Redis
Databases: Postgres and NoSQL (Elasticsearch, MongoDB, Cassandra)
Infrastructure: Kubernetes, microservices, resource management
Cloud: AWS preferred; GCP or Azure acceptable
Frontend: Working knowledge of HTML and JavaScript
Bonus: React, HTMX, automation tools, or consumer privacy industry experience
'''

system_prompt = SystemMessagePromptTemplate.from_template('You are an AI Career Expert. You Review Resumes and Help To Tell People What To Learn To Be Relevant in The AI Era')

user_prompt = HumanMessagePromptTemplate.from_template('''
You are tasked with giving career advice based on a resume. 
The resume is found here {resume}

The career advice should be very specific to the resume submitted.
Ensure you answer is always limited to 100 words only
The advice should show skills, technologies and 3 possible job roles the owner of the resume should get to be relevant in the AI Era.
Only output the career advice and no other explanation or text can be provided.
''', 
                                            input_variable='resume')

The next thing is for us to check if the input variable actually works

In [12]:
user_prompt.format(resume='Software Engineer With 6 Years Of Experience')

HumanMessage(content='\nYou are tasked with giving career advice based on a resume. \nThe resume is found here Software Engineer With 6 Years Of Experience\n\nThe career advice should be very specific to the resume submitted.\nEnsure you answer is always limited to 100 words only\nThe advice should show skills, technologies and 3 possible job roles the owner of the resume should get to be relevant in the AI Era.\nOnly output the career advice and no other explanation or text can be provided.\n', additional_kwargs={}, response_metadata={})

Since we have the system and user prompts, the next thing for us to do is to create the chat prompt

In [13]:
from langchain_core.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# Test Out What Our First Prompt Is
print(first_prompt.format(resume='Software Engineer With 6 Years Of Experience'))

System: You are an AI Career Expert. You Review Resumes and Help To Tell People What To Learn To Be Relevant in The AI Era
Human: 
You are tasked with giving career advice based on a resume. 
The resume is found here Software Engineer With 6 Years Of Experience

The career advice should be very specific to the resume submitted.
Ensure you answer is always limited to 100 words only
The advice should show skills, technologies and 3 possible job roles the owner of the resume should get to be relevant in the AI Era.
Only output the career advice and no other explanation or text can be provided.



The next thing is for us to combine the first_prompt we have with the llm we created to an LLM Chain

## What is An LLM Chain?
This is a simple step-by-step work flow where you drop and input and the input is sent to an LLM via a prompt. The LLM's out put is then formatted, transformed, passed into another step or another LLM.

In [14]:
chain_one = (
    {'resume': lambda x: x['resume']}
    | first_prompt
    | creative_llm
    | {'final_advice': lambda x : x.content}
)

Now we have created the Lang Chain Expression Language. Let Us Invoke It

In [ ]:
advice = chain_one.invoke({'resume': resume})